In [ ]:
# Full Vehicle Classification Training Pipeline using MobileNetV2
# Includes augmentation, class weighting, fine-tuning, and saving as .keras

from google.colab import drive
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import shutil







In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = '/content/drive/MyDrive/vehicle_classification/datasets/'
save_path = '/content/drive/MyDrive/vehicle_classification/processed_images/'
split_path = '/content/drive/MyDrive/vehicle_classification/vehicle_classification_datasets/'
base_dir = '/content/drive/MyDrive/vehicle_classification'
processed_path = os.path.join(base_dir, 'processed_images')
split_path = os.path.join(base_dir, 'vehicle_classification_datasets')
log_dir = os.path.join(base_dir, 'logs_mobilenet')
model_path = os.path.join(base_dir, 'vehicle_classifier_mobilenet2.keras')



In [ ]:
if not os.path.exists(save_path):
    os.makedirs(save_path)

#Load data with resizing and rescaling
datagen = ImageDataGenerator(rescale=1./255)
data = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 3130 images belonging to 13 classes.


In [ ]:
img_count = 0
for images, labels in data:
    for i in range(images.shape[0]):
        img = images[i] * 255.0
        img = img.astype(np.uint8)
        pil_img = array_to_img(img)

        class_index = np.argmax(labels[i])
        class_name = list(data.class_indices.keys())[class_index]

        class_dir = os.path.join(save_path, class_name)
        os.makedirs(class_dir, exist_ok=True)

        img_filename = f"{class_name}_{img_count:04d}.png"
        img_path = os.path.join(class_dir, img_filename)
        pil_img.save(img_path)
        img_count += 1

    if img_count >= data.samples:
        break

print(f"Saved {img_count} resized and normalized images to: {save_path}")

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Saved 3130 resized and normalized images to: /content/drive/MyDrive/vehicle_classification/processed_images/


In [ ]:
#  Create split folders
for split in ['train', 'val', 'test']:
    for cls in os.listdir(processed_path):
        os.makedirs(os.path.join(split_path, split, cls), exist_ok=True)

#  Split the processed dataset
for cls in os.listdir(processed_path):
    class_dir = os.path.join(processed_path, cls)
    images = os.listdir(class_dir)

    train_val, test = train_test_split(images, test_size=0.15, random_state=42)
    train, val = train_test_split(train_val, test_size=0.176, random_state=42)  # 0.176 * 0.85 ≈ 15%

    for split_name, split_files in zip(['train', 'val', 'test'], [train, val, test]):
        for img_file in split_files:
            src = os.path.join(class_dir, img_file)
            dst = os.path.join(split_path, split_name, cls, img_file)
            shutil.copy(src, dst)

print(" Dataset split into train, val, and test folders")

NameError: name 'processed_path' is not defined

In [ ]:
for folder in ['train', 'val', 'test']:
    path = os.path.join(split_path, folder, 'class_13')
    shutil.rmtree(path, ignore_errors=True)


In [ ]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    shear_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.5, 1.5]
)
val_test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    os.path.join(split_path, 'train'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
val_data = val_test_gen.flow_from_directory(
    os.path.join(split_path, 'val'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
test_data = val_test_gen.flow_from_directory(
    os.path.join(split_path, 'test'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 4137 images belonging to 12 classes.
Found 891 images belonging to 12 classes.
Found 891 images belonging to 12 classes.


In [ ]:
#  Compute class weights
y_train = train_data.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

In [ ]:
#  Build model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True
for layer in base_model.layers[:-100]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Callbacks
callbacks = [
    TensorBoard(log_dir=log_dir),
    ModelCheckpoint(model_path, save_best_only=True, monitor='val_loss', mode='min'),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]


In [ ]:
#  Train model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=75,
    class_weight=class_weights,
    callbacks=callbacks
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/75
130/130 ━━━━━━━━━━━━━━━━━━━━ 1774s 13s/step - accuracy: 0.0862 - loss: 2.7220 - val_accuracy: 0.1515 - val_loss: 2.3955
Epoch 2/75
130/130 ━━━━━━━━━━━━━━━━━━━━ 499s 4s/step - accuracy: 0.1587 - loss: 2.4252 - val_accuracy: 0.2974 - val_loss: 2.1525
Epoch 3/75
130/130 ━━━━━━━━━━━━━━━━━━━━ 472s 4s/step - accuracy: 0.2771 - loss: 2.2100 - val_accuracy: 0.3793 - val_loss: 1.9301
Epoch 4/75
130/130 ━━━━━━━━━━━━━━━━━━━━ 476s 4s/step - accuracy: 0.3441 - loss: 2.0550 - val_accuracy: 0.4602 - val_loss: 1.7155
Epoch 5/75
130/130 ━━━━━━━━━━━━━━━━━━━━ 470s 4s/step - accuracy: 0.4211 - loss: 1.8659 - val_accuracy: 0.5309 - val_loss: 1.5304
Epoch 6/75
130/130 ━━━━━━━━━━━━━━━━━━━━ 471s 4s/step - accuracy: 0.4944 - loss: 1.6903 - val_accuracy: 0.5915 - val_loss: 1.3784
Epoch 7/75
130/130 ━━━━━━━━━━━━━━━━━━━━ 474s 4s/step - accuracy: 0.5330 - loss: 1.6031 - val_accuracy: 0.6274 - val_loss: 1.2572
Epoch 8/75
130/130 ━━━━━━━━━━━━━━━━━━━━ 506s 4s/step - accuracy: 0.5722 - loss: 1.4291 - val_ac

In [ ]:

model = load_model('/content/drive/MyDrive/vehicle_classification/vehicle_classifier_mobilenet.keras')

test_data = val_test_gen.flow_from_directory(
    os.path.join(split_path, 'test'),
    target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=False
)
loss, acc = model.evaluate(test_data)
print(f"Test Accuracy: {acc:.4f}")

#  Predict single image with unknown fallback
class_labels = sorted(os.listdir(os.path.join(split_path, 'train')))

def predict_vehicle_image(img_path, model, class_labels, threshold=0.6):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array, verbose=0)
    confidence = np.max(prediction)
    class_idx = np.argmax(prediction)

    if confidence < threshold:
        label = "Unknown Vehicle"
    else:
        label = f"{class_labels[class_idx]} ({confidence*100:.1f}%)"

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {label}")
    plt.show()
    return label